# Étape 1 : Prérequis

In [ ]:
# Vérification du GPU et installation des dépendances nécessaires
!nvidia-smi
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers


In [ ]:
# Afficher les versions des bibliothèques installées
!nvidia-smi
!echo "Version de bitsandbytes :"
!pip show bitsandbytes
!echo "Version de peft :"
!pip show peft
!echo "Version de trl :"
!pip show trl
!echo "Version de accelerate :"
!pip show accelerate
!echo "Version de datasets :"
!pip show datasets
!echo "Version de transformers :"
!pip show transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configuration pour la quantification en 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [ ]:
# Si vous utilisez Google Colab, connectez-vous à Hugging Face
from huggingface_hub import notebook_login
notebook_login()

# Étape 2 : Chargement du modèle
Chargement du modèle avec la configuration de quantification QLoRA pour réduire l'utilisation de la mémoire

In [ ]:
model_id = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

In [ ]:
# Fonction pour générer une réponse à partir du modèle
def obtenir_reponse(query: str, model, tokenizer) -> str:
    device = "cuda:0"

    template_prompt = """
    <start_of_turn>utilisateur
    Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète correctement la demande.
    {query}
    <end_of_turn>\n<start_of_turn>modèle
    """
    prompt = template_prompt.format(query=query)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded

In [ ]:
# Exemple de test du modèle
resultat = obtenir_reponse(query="écrire des tests unitaires pour cette fonction def add(a,b): return a+b en utilisant pytest", model=model, tokenizer=tokenizer)
print(resultat)

# Étape 3 : Chargement du dataset pour le fine-tuning



Nous utiliserons ce [dataset](https://huggingface.co/datasets/TokenBender/code_instructions_122k_alpaca_style) qui est un excellent point de départ pour entraîner des modèles de génération de code.

In [ ]:
from datasets import load_dataset

# Charger les données à partir du fichier data.json
dataset = load_dataset("json", data_files="/content/data.json", split="train")
print(dataset)

In [ ]:
# Transformation des données en prompts adaptés à l'entraînement
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a pytest unit test function that ' \
                  'appropriately completes the request.\n\n'
    # Samples with additional context info.
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} Here is the function:\n{data_point["input"]} <end_of_turn>\n<start_of_turn>model\n{data_point["output"]} <end_of_turn>"""
    # Without additional context
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model\n{data_point["output"]} <end_of_turn>"""
    return text


In [ ]:
# Ajouter la colonne "prompt" au dataset
text_column = [generer_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [ ]:
# Mélanger et diviser le dataset en ensembles d'entraînement et de test
dataset = dataset.shuffle(seed=1234)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]
print(train_data)
print(test_data)

# Étape 4 : Application de LoRA (Low-Rank Adaptation)
Application de la technique LoRA pour optimiser l'entraînement du modèle


In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

# Activation du gradient checkpointing pour optimiser la mémoire
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import bitsandbytes as bnb

# Identification des modules linéaires du modèle pour LoRA
def trouver_tous_noms_lineaires(model):
    cls = bnb.nn.Linear4bit 
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if 'lm_head' in lora_module_names: 
            lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [ ]:
modules = trouver_tous_noms_lineaires(model)
print(modules)

In [ ]:
# Configuration de LoRA
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Application de la configuration LoRA au modèle
model = get_peft_model(model, lora_config)
trainable, total = model.get_nb_trainable_parameters()
print(f"Paramètres entraînables : {trainable} | total : {total} | Pourcentage : {trainable/total*100:.4f}%")


# Étape 5 : Entraînement du modèle
Fine-tuning du modèle avec qLora et Supervised Fine-Tuning (SFT)

In [ ]:
import transformers
from trl import SFTTrainer

# Configuration de l'entraîneur
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [ ]:
# Lancement de l'entraînement
model.config.use_cache = False  # silence les avertissements. Veuillez réactiver pour l'inférence !
trainer.train()

In [ ]:
# Partage des adaptateurs sur le Hub Hugging Face
nouveau_modele = "model-aftaitesting" # Nom du modèle que vous allez pousser sur le Hub Hugging Face
trainer.model.save_pretrained(nouveau_modele)


In [ ]:
# Fusion et sauvegarde du modèle
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model = PeftModel.from_pretrained(base_model, nouveau_modele)
merged_model = merged_model.merge_and_unload()

In [ ]:
# Sauvegarder le modèle fusionné
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

In [ ]:
# Pousser le modèle et le tokenizer sur le Hub Hugging Face
merged_model.push_to_hub(nouveau_modele, use_temp_dir=False)
tokenizer.push_to_hub(nouveau_modele, use_temp_dir=False)

In [ ]:
# Tester le modèle fine-tuné
resultat = obtenir_reponse(query="écrire simplement du code de tests unitaires en utilisant pytest pour cette fonction : def add(a,b): return a+b avec pytest. Donner juste le code, sans instruction", model=merged_model, tokenizer=tokenizer)
print(resultat)

In [ ]:
import torch
import transformers
from trl import SFTTrainer
import matplotlib.pyplot as plt
from datasets import load_metric

# Étape 1 : Calcul de l'accuracy
def calculer_accuracy(predictions, labels):
    """Calcule l'accuracy en comparant les prédictions avec les étiquettes.
    
    :param predictions: torch.Tensor: Prédictions du modèle
    :param labels: torch.Tensor: Étiquettes réelles
    :return: float: Accuracy
    """
    predictions = predictions.argmax(dim=-1)
    accuracy = (predictions == labels).float().mean()
    return accuracy.item()


In [ ]:
# Configuration de l'entraîneur
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

accuracy_metric = load_metric("accuracy")

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        evaluation_strategy="epoch",  # Effectuer une évaluation après chaque epoch
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [ ]:
# Listes pour stocker l'accuracy après chaque epoch
train_accuracies = []
eval_accuracies = []

In [ ]:
# Modification de la fonction d'entraînement pour enregistrer l'accuracy
for epoch in range(trainer.args.num_train_epochs):
    # Entraînement
    trainer.train()
    
    # Évaluation
    eval_results = trainer.evaluate()
    
    # Calcul de l'accuracy sur l'ensemble d'entraînement et de test
    train_accuracy = calculer_accuracy(eval_results['predictions'], eval_results['label_ids'])
    eval_accuracy = calculer_accuracy(eval_results['eval_predictions'], eval_results['eval_label_ids'])
    
    train_accuracies.append(train_accuracy)
    eval_accuracies.append(eval_accuracy)
    
    print(f"Epoch {epoch + 1}: Accuracy Entraînement = {train_accuracy:.4f}, Accuracy Évaluation = {eval_accuracy:.4f}")


In [ ]:
# Étape 2 : Tracer les courbes d'accuracy
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Accuracy Entraînement')
plt.plot(range(1, len(eval_accuracies) + 1), eval_accuracies, label='Accuracy Évaluation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Courbes d\'Accuracy au cours de l\'Entraînement')
plt.legend()
plt.grid(True)
plt.show()